# String Demo
## Connecting to Redis

The first step in working with Redis, like most any database, is setting 
up a connection to the database server.  The server could be running 
on the same machine as the application program or on a remote machine
dedicated to running the database.

This chapter will walk through establishing a connection to Redis on either a local or
remote host, and using that connection to implement your first Redis program. To
accomplish these tasks, you will need to learn about:

* Redis client libraries
* Database connection parameters
* Establishing connections to a database

After learning the basics of establishing a connection, we will walk through a
Redis version of the classic Hello World program.

## Client Libraries 

Redis is accessed through a client object that provides methods for executing the Redis
commands. This paradigm of exposing a client object is used by most Redis libraries as
well as a host of other NoSQL and relational database libraries. This paradigm is popular
across a programming languages.

The most popular client library for Python is the open source `redis-py` package. The
`redis-py` package can be installed on a system using any of the popular Python package
management systems. We will be using this library for all of our examples.

Creating a Redis client with `redis-py` is extremely simple - just instantiate an instance
of the `StrictRedis` object. Once constructed, the resulting instance allows you to
manipulate data on your Redis server through method calls.

* * *

> **Note**
>
> For historical reasons, the `redis-py` API provides two client objects 
> `Redis` and `StrictRedis`.  `StrictRedis` should be used for all new projects.
> The `Redis` class is provided for backwards compatibility with older versions
> of the library.
> 

* * *


## Connection Parameters

Connection parameters are used to identify the Redis database your program will be
communicating with.  Although that database could be on the same machine as your
program or a remote server, the parameters used to specify a connection are always
the same.

There are four connection parameters used with all Redis clients: `host`, `port`,
`password`, and `db`.  Each of the parameters is documented in the table below:

Paramter | Meaning | Examples
---------|---------|----------
host | DNS name of database server | redis
port | port number of database     | 6379 (default Redis port)
password | database password | secret
db | virtual database number | 0 (default)

The first three parameters: `host`, `port`, and `password` should be familiar to anyone
with client-server programming experience.  These parameters are used to identify the 
network address of a sever process; however, the `db` parameter is unique to Redis.

In Redis, your data can be partitioned across multiple *virtual* databases
identified by an integer.  When specifying a database connection, the `db` parameter is
used to specify which virtual database your connection will be interacting with.  If
unspecified the default database is zero. Virtual databases an be used to partition your data 
within the same process.  

* * *

> **Tip**
>
> While Redis supports multiple virtual databases to separate your data, the use of 
> multiple databases in production systems is discouraged. The recommended practice for 
> data separation in Redis is to deploy multiple database processes.  Some systems, 
> including Redis<sup>e</sup>, have gone as far as dropping support for multiple databases
> given how strongly the practice is discouraged.
>

* * *

In the example below, we show how to create a single connection to Redis using the
`StrictRedis` object. We specify the parameters for our database connection in a Python
dictionary and pass the dictionary to the `StrictRedis` constructor using the `**` syntax
to unpack the dictionary.

The constructor for both the `StrictRedis` and `Redis` objects takes a number of keyword
arguments to specify a Redis connection. The `**` syntax conveniently allows us to specify
our connection configuration once as a dictionary and then reuse that dictionary structure
whenever we need to specify connection parameters.



## Hello World

Hello World is the traditional program used to introduce new technologies, so 
why not our Redis training with Hello World as well.  Since the key-value operations
are one of the most popular ways to use Redis, it seems only fitting to start with 
the Redis GET and SET commands to build our Hello World program.

Our Redis Hello World program demonstrates several key elements used in almost all Redis programs.  Our first Redis program will show how to:

* Import our client library interface 
* Connect to a Redis database 
* Load our Hello World message into Redis
* Fetch our message from the database
* Print the results from the database as output


*Run this sample program by selecting the Notebook code cell and pressing SHIFT + ENTER 
to execute.*

In [ ]:
import redis

# example connection parameters 
config = {
    "host": "redis",
    "port": 6379
}

r = redis.StrictRedis(**config)

r.set("workshop_message", "Hello World!")
message = r.get("workshop_message").decode('utf-8')

print (message)


# Tracking Votes with Redis Strings

This chapter will took at using the Redis string datatype to track user votes on a
website. User voting can take many forms: up/down votes, thumbs up, likes, but essentially
all of these experiences distill down to recording positive and negative impressions
from the user in the database.

This chapter will look at using the Redis string datatype to maintain a global count of
votes on a particular blog post. To developers unfamiliar with Redis, maintaining a vote
count using a string might sound incorrect, but the Redis String type is different from
the string types in other programming languages you are familiar with.

In this chapter, we will learn about:

* Storing data in Redis strings
* Manipulating data in Redis strings
* How Redis strings differ from string datatypes 
* Working with bitmaps, integers and floating points

At the end of the chapter, you should understand how to use the Redis string datatype to
implement key-value operations and apply Redis string operations to a variety of development problems.

## Strings

Redis provides a string datatype that is used to associate data with a particular key.
Redis strings are the most basic datatype available in Redis and one of the first datatypes
that users learn to work with.  Our Hello Work program in the chapter on connecting to Redis used the string datatype.

Most programmers associate string datatypes with text data like the string datatype in
most modern programming languages.  Redis strings are more like buffers that can store a wide range of data.  Redis strings can be used to store:

* Integers
* Floating point numbers
* Bitmaps
* Strings
* Binary objects

String values in Redis can be up to 512 MB in size, allowing you to store substantial binary objects including images, sound files, and object files.

* * *

> **Note**
>
> Redis keys are also binary safe, so although most programming examples show Redis keys
> as strings, you can use any binary sequence as a key in Redis.  Keys can also be up
> 512 MB in length, but we caution against using very long keys as they can have an
> adverse effect on performance.  Standard practice in Redis is to use a structured key
> names with a schema, such as `comment:1234:reply.to` or `comment:1234:reply-to`.
>
> We also recommend that you do not use very short names for Redis keys either, ease of 
> use and clarity needs to be balanced out against performance.  The performance 
> improvements of a key like `c1234r2` versus `comment:1234:reply.to` are minimal and
> don't outweigh the loss of clarity.
>

* * *

The current version of Redis (3.2) provides 24 different commands to work with the string
data type.  Redis provides commands to:

* Read and write string values
* Manipulate strings as integers
* Manipulate strings as floating point numbers
* Use strings to store bitmaps

Redis allows you several ways to perform calculations on your data *without* having to read
the data into your application, manipulate it, and then write it back to the server.

## Tracking Votes

In our examples, votes will be integer values, both positive and negative, associated with
a particular blog post. Our system will track raw votes, each user can either vote up or
down for a given post. The resulting vote count is displayed to users as a way of
prioritizing posts to read.

## Data Storage Conventions

Our votes will be stored in Redis using the string datatype. One string will be stored for
each item under the key `item:{item_no}:vote-count` and the string will be used to store
an integer vote count. An up-vote (positive vote) is tracked as a `+1` and a down-vote
(negative vote) is tracked as a `-1`. The votes for a particular item are the sum of all
the up and down votes.

To improve the clarity of our code, the Notebook environment includes a few utility
functions, allowing you to focus on the Redis implementation details. In this chapter, the
main functions that will be used from the `workshop` package are:

* `item_vote_key` - generates the proper key from an item id
* `create_item_vote` - clears current vote totals from the database
* `get_item_vote` - returns the vote total for an item 
* `users_vote_key` - returns the key for tracking a users vote
* `clear_users_vote` - clears a users voting record

## Vote Counting

### Counting an Upvote

When a user views a blog post on our website, they are given the opportunity to vote on the
post.  If the user thought the post was a good post and want to recommend the post to
others, they can upvote the post.  In our first version of counting upvotes, we aren't going to worry about duplicates, so we need code that will:

* Generate an appropriate Redis key
* Increment item vote count

This can be accomplished with a single Redis command - INCR.

The INCR (**INCR**ement) command operates on an integer value stored in Redis as a string
and increments the value of the key on the server by one.

In [ ]:
def item_vote_key(item_id):
    "Returns the Redis key for maintaining item vote counts"
    
    return "item:" + str(item_id) + ":vote-count"

def upvote_item(r, item_id):
    "Upvotes an item and stores it in the Redis database"
    
    # get our key
    key = item_vote_key(item_id)
 
    return r.incr(key)

# execute our example
print("Vote count for item 20: " + str(upvote_item(r,20)))

In [ ]:
def downvote_item(r, item_id):
    "Downvotes an item and stores it in the Redis database"
    
    # get our key
    key = item_vote_key(item_id)
 
    return r.decr(key)

# execute our example
print ("Vote count for item 20: " + str(downvote_item(r, 20)))


### Verify numbers
* use redisinsight to verify created keys and values

### Tracking Votes

Now that we've seen a couple examples of vote recording functions, we need to flesh out or
implementation to demonstrate a more realistic approach to the problem. Most websites
don't allow users to vote more than once on a given item. In this iteration of the code,
we will update our upvote and downvote methods to check that a user has not already
registered a vote on a particular item. A user's vote will only be applied once, otherwise
the function will ignore the vote and return `False`.

We will be using the Redis bitmap "type" to maintain a record of votes. As we saw earlier,
Redis has commands that treat strings as integers and performs basic arithmetic on the
values. Redis also has commands that treat strings as bitmaps and can perform standard bit
operations: test, set, and logical. Redis represents bitmaps as a variable width vector of
bits with a zero-based indexing scheme. Bitmaps can grow up to Redis' maximum value size
(512 MB).

We can augment our original functions with two additional commands, GETBIT and SETBIT, to
record votes using Redis' bitmaps. In our code, we will store the users votes in the key
`user:{user_id}:votes` with a bitmap indexed by the item id. A zero value in the bitmap
will indicate no vote and a one value in the bitmap will indicate the user voted.

To prevent the user from having a duplicate vote we need to:

* Lookup the users vote for a particular item_id
* Check for a recorded vote
* Record that the user voted
* Apply the value of the users vote

The example code below shows a modified version of our original voting functions with
duplicate checks added.

In [ ]:
def get_item_vote(r, item_id):
    "Returns the item vote count"

    key = item_vote_key(item_id)
    res = r.get(key)
    if res is None:
        return "0"
    else:
        return res
    
def users_vote_key(user_id):
    "Returns the key for tracking a users votes"

    return "user:" + str(user_id) + ":votes"

def user_voted(r, user_id, item_id):
    "Checks if a user has already voted on an item"
    key = users_vote_key(user_id)
    voted = r.getbit(key, item_id)
    
    # voted is returned as a bit (0/1), cast to Python boolean
    return bool(voted)
    
    
def upvote_item(r, user_id, item_id):
    "Processes a user's upvote and stores it in Redis (with duplicate checking)"
    
    if not user_voted(r, user_id, item_id):
        r.setbit(users_vote_key(user_id), item_id, 1)
        r.incrby(item_vote_key(item_id), 2)
        print("user: " + str(user_id) + " upvote counts for item " + str(item_id))
        return True
    else:
        print("user:" + str(user_id) + " previously voted, downvote not counted for item " + str(item_id))
        return False
        

def downvote_item(r, user_id, item_id):
    "Processes a user's downvote and stores it in Redis (with duplicate checking)"
    
    if not user_voted(r, user_id, item_id):
        r.setbit(users_vote_key(user_id), item_id, 1)
        r.decr(item_vote_key(item_id))
        print("user: " + str(user_id) + " downvote counts for item " + str(item_id))
        return True
    else:
        print("user:" + str(user_id) + " previously voted, downvote not counted for item " + str(item_id))
        return False
    
upvote_item(r, 3001, 20)
upvote_item(r, 3001, 21)
upvote_item(r, 3001, 20)

downvote_item(r, 3002, 20)
downvote_item(r, 3002, 20)

print ("Vote count for item 20: " + str(get_item_vote(r, 20)))
print ("Vote count for item 21: " + str(get_item_vote(r, 21)))
print ("Vote count for item 22: " + str(get_item_vote(r, 22)))


In [ ]:
def clear_users_vote(r, user_id):
    "Clears a users vote history"

    key = users_vote_key(user_id)
    return r.delete(key)

def clear_item_vote(r, item_id):
    "Clears the item vote count"

    key = item_vote_key(item_id)
    return r.delete(key)

clear_item_vote(r,20)
clear_item_vote(r,21)
clear_users_vote(r,3001)
clear_users_vote(r,3002)
upvote_item(r, 3001, 20)
upvote_item(r, 3001, 21)
upvote_item(r, 3001, 20)

downvote_item(r, 3002, 20)
downvote_item(r, 3002, 20)
print ("Vote count for item 20: " + str(get_item_vote(r, 20)))
print ("Vote count for item 21: " + str(get_item_vote(r, 21)))
print ("Vote count for item 22: " + str(get_item_vote(r, 22)))

### verify in CLI

* getbit user:3001:votes 10
* getbit user:3001:votes 20
* getbit user:3001:votes 22
* getbit user:3001:votes 10
* getbit user:3001:votes 20
* getbit user:3001:votes 22


### verify TTL
* setex ryan 10 istheman
* get ryan
* ttl ryan
* ttl ryan
* get ryan